In [ ]:
import cvxpy as cvx # A good library with open-source solvers and modeling language
import numpy as np # Basic library for array structures
from numpy.random import multivariate_normal, randn, uniform, choice # Probability distributions
from scipy.linalg import norm # Efficient norm calculation
from scipy.linalg import solve # Linear system solve
from math import sqrt
import time # Measure elapsed time
import sys

np.set_printoptions(threshold=sys.maxsize)


### Basic parameters

In [ ]:
n = 13
p = 6
N = p * n**2
c = 5


number_of_offices = np.array([3, 3, 3, 2, 2])
offices_per_wing = np.block([np.array([0]) ,np.cumsum(number_of_offices)])

colors=["Unaffected", "MIDO", "LAMSADE", "Student Association", "Presidency"]

Helper functions

In [ ]:
def office_in_wing(office_number, wing_number, offices_per_wing):
  return office_number >= offices_per_wing[wing_number] and office_number < offices_per_wing[wing_number+1]

In [ ]:
# TODO - update so that it can work with Y constraints as well
# rownb - list of indices to pick
def get_rows(phase, rownb, colDim = n, rowDim = n):
  """ Creates a vector of 1's to be used as vector of coefficients

        Parameters
        ----------
        phase : integer number in range [0, 5]
                for which phase the coeffs will be taken
        rownb:  list of integers

        Returns
        -------
        np array
            vectore of coefficients
        """
  # zeros for previous phases
  p1 = np.zeros(phase * colDim * rowDim)
  # for all the rows corresponding to an office of the wing renowated in this phase, fill it with 1's
  p2 = np.ravel(np.array([[1 if j in rownb else 0 for j in range(rowDim)] for _ in range(colDim)]).T)
  p3 = np.zeros((p - phase - 1) * colDim * rowDim)

  return np.block([p1, p2, p3])



In [ ]:
# colnb - list of indices to pick up
def get_columns(phase, colnb,  colDim = n, rowDim = n):
  p1 = np.zeros(phase * colDim * rowDim)
  p2 = np.ravel(np.array([[1 if j in colnb else 0 for j in range(rowDim)] for _ in range(colDim)]))
  p3 = np.zeros((p - phase - 1) * colDim * rowDim)

  return np.block([p1, p2, p3])
# print(get_columns(0, [0, 1, 2])[:n**2].reshape(n, n))

In [ ]:
def within_epsilon(x,y, epsilon=0.0001): return abs(x-y)<=epsilon

### Constraint 1
The initial assignment is respected

In [ ]:
cstrInit = np.zeros((n, n))

for i in range(3) :
  cstrInit[i] = np.zeros(n)

for i in range(3, n):
  vec = [1 if j == i else 0 for j in range(n)] #grab diagonal elements
  cstrInit[i] = vec

cstrInit = np.ravel(cstrInit)

b = np.array([10])
A = np.block([cstrInit, np.zeros((p - 1) * n**2)])

### Constraint 2
No more than one office move to the same destination.
Sum of the entries along the columns corresponding to the given phase is fixed to 0.

In [ ]:
# Nobody goes to a wing under construction

constraint1 = np.zeros((p - 1, N))
for i in range(p - 1):
  constraint1[i] = get_columns(i, range(offices_per_wing[i], offices_per_wing[i + 1]))

b = np.block([
    b,
    np.zeros(p - 1)])
A = np.block([[A],
              [constraint1]])

print(A.shape, b.shape)

(6, 1014) (6,)


### Constraint 3
No more than one office move to the same destination.
All columns of x upper-bounded by 1

In [ ]:
constraint5 = np.zeros((p * n, N))
for phase in range(p):
  for office in range(n):
    constraint5[phase*n + office] = get_columns(phase, [office])

B = constraint5

### Constraint 4
Occupied offices move somewhere and nothing comes out of an empty
office.

Essentially, phase to phase correspondance

#### Constraint 4.1
Occupied offices move somewhere.

In [ ]:

# If someone was here, move
# Otherwise nothing should come out of an empty office

# There are correspondences - 1->0, 2->1, 3->2, 4->3

constraint3 = np.zeros(((p - 2) * n, N))
for phase in range(1, p - 1):
  for office in range(n):
    if office in range(offices_per_wing[phase], offices_per_wing[phase + 1]): continue
    constraint3[(phase - 1) * n + office] -= get_columns(phase - 1, [office])

    if office in range(offices_per_wing[phase], offices_per_wing[phase + 1]) \
    or phase == p - 1:
      constraint3[(phase - 1) * n + office] += get_rows(phase, [office])

    else:
      constraint3[(phase - 1) * n + office,
                  phase * n ** 2 + n * office + office] += 1

  A = np.block([[A],
                [constraint3]])
  b = np.block([b, np.zeros((p - 2) * n)])

k = 2
l = 2
#print(constraint3[k][0 * n ** 2 : (0 + l) * n ** 2].reshape((l * n, n)))


#### Constraint 4.2 - office not concerned

The reasoning here is that, if the wing is untouched by the renovation, then no changes in entries of $X$ corresponding to them should happen. It is enough to constrain the diagonals, since other entries will be zeroes by the virtue of minimization.

In [ ]:

constraint3 = np.zeros(((p - 2), N))

for i in range(1, p - 1) :
  # NB: there is a bijection between wings and phases[1:], so we can index
  # "offices_per_wing" with "phase" value

  # add coefficients of the rows of the wing in the current phase
  constraint3[i - 1] += get_rows(i, range(offices_per_wing[i],
                                         offices_per_wing[i + 1]))
  # subtract coefficients of the rows of the wing in the previous phase
  constraint3[i - 1] -= get_columns(i - 1, range(offices_per_wing[i],
                                                offices_per_wing[i + 1]))

  A = np.block([[A],
                [constraint3]])
  b = np.block([b, np.zeros((p - 2))])

# print(A.shape, b.shape)

k = 0
l = 2
# print(constraint3[k][k * n ** 2 : (k + l) * n ** 2].reshape((l * n, n)))
# print(constraint3[k][0 * n** 2 : 5 * n ** 2].reshape((5 * n, n)))


### Constraint 5
The final assignment is respected

In [ ]:
# Empty offices in the last phase - wing A and B3
emptyFinal = np.block([
    np.zeros((p - 1) * n**2), # Previous  phases

    np.zeros(3 * n), #Wing N

    # np.ravel(np.array([[1 if j == 2 else 0 for j in range(n)] for _ in range(3)])), # Wing B

    np.zeros(2 * n),

    np.ones(n),

    np.zeros( (3 + 2) * n), # Wings P and C

    np.ones(2 * n) # Wing A
])

A = np.block([[A],
              [emptyFinal]])
b = np.block([b, 0])



# Occupied offices in the last phase
occupiedFinal = np.zeros((n - 3, N))
for i in range(n):
  if i in [5, 11, 12] : continue # Empty offices

  cst = np.block([
      np.zeros((p - 1) * n**2),
      np.ravel([[1 if j == i else 0 for j in range(n)] for _ in range(n)])
  ])

  row_number = i - (i > 5)
  occupiedFinal[row_number] = cst


b = np.block([b,
              0,
              np.ones(n - 3)])
A = np.block([ [A],
              [emptyFinal],
              [occupiedFinal]
               ])


### Constraint 6 -
Office moved to at most one destination. Each row upperbounded by 1

In [ ]:
constraint5 = np.zeros((p * n, N))
for phase in range(p):
  for office in range(n):
    constraint5[phase*n + office] = get_rows(phase, [office])

D = constraint5

In [ ]:
print(A.shape, b.shape)

(18, 1014) (18,)


## CVX Resolution

In [ ]:

print(A.shape, b.shape)
np.set_printoptions(threshold=sys.maxsize)
# print(B.shape, f.shape)

x = cvx.Variable((N))

cobj = np.ones((N))
ignorevalues = np.ravel([np.eye(n) for _ in range(p)])
cobj -= ignorevalues

objective = cvx.Minimize(cobj @ x)


constraints = [A@x == b,
               B @ x <= 1,
               D @ x <= 1,
               x <= 1,
               x >= 0]
prob = cvx.Problem(objective, constraints)

result= prob.solve()
print(prob.status)

values = abs(
    np.reshape(
    np.round(x.value, 2),(p,n,n)))
print(values)

# np.savetxt("solution", values.reshape(p * n, n), fmt='%1.2g')

(18, 1014) (18,)
optimal
[[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]

 [[0.38 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.  

## Colors

### Y constraints

In [ ]:
N2 = n * c * p

Ay = np.zeros(((p + 1) * n * c, p * n * c))

In [ ]:
# Initial color assignment constraint
InitAy = np.block([np.eye(n * c), np.zeros((n * c, (p - 1) * n * c))])
# Final color assignment constraint
FinalAy = np.block([np.zeros((n * c, (p-1) * n * c)), np.eye(n * c)])


miniAy = np.diag(np.block([
    np.zeros(n * c),
    np.ones(n * c * (p-1))
    ]))

Aw = np.empty((n * c * (p - 1), n**2 * c * p))
for k in range(1,p):
  for i in range(n):
    for j in range(c):
      index1 = k * n**2 * c + i * n * c + j * n
      Aw[(k - 1) * n * c + i * c + j] = np.block([
          np.zeros(index1),
          np.ones(n),
          np.zeros(n**2 * c * p - index1 - n)
      ])

Aw = np.block([
    [np.zeros((n * c, n**2 * c * p))],
    [Aw]
    ])

#### Constraint 1
Initial colors.

In [ ]:
constraint0 = np.zeros((n, c))

coeffs = [(3, 1), (4, 2), (5, 1), (6, 3), (7, 3), (8, 2), (9, 4), (10, 4), (11, 2), (12, 1)]

for i, j in coeffs:
  constraint0[i, j] = 1

init_colors = np.ravel(constraint0)

#### Constraint 5

Final colors.

In [ ]:
constraint0 = np.zeros((n, c))

coeffs = [(0, 4), (1, 4), (2, 1), (3, 1), (4, 1), (6, 2), (7, 2), (8, 2), (9, 3), (10, 3)]

for i, j in coeffs:
  constraint0[i, j] = 1

final_colors = np.ravel(constraint0)

In [ ]:
x = cvx.Variable((N))

N2 = n*c*p
y = cvx.Variable((N2))

w = cvx.Variable((n * N2))

In [ ]:
def vectorize_index(indexlist, vartype):
  if vartype =="x" : indexes_lengths = np.array([n**2, n, 1])
  elif vartype=="y" : indexes_lengths = np.array([n * c, c, 1])
  elif vartype=="w" : indexes_lengths = np.array([c * n**2, n * c, n, 1])

  return np.array(indexlist) @ indexes_lengths

#### W constraints

In [ ]:
wconstraints =[w >= 0, w <= 1]

for k in range(1,p):
  for i in range(n):
    for j in range(c):
      for e in range(n):
        wconstraints.extend([
          w[vectorize_index([k,i,j,e], "w")] <= x[vectorize_index([k,e,i], "x")],
          w[vectorize_index([k,i,j,e], "w")] <= y[vectorize_index([k-1,e,j], "y")],
          w[vectorize_index([k,i,j,e], "w")] >= y[vectorize_index([k-1,e,j], "y")] + x[vectorize_index([k,e,i], "x")] - 1
        ])

## Solving

In [ ]:
yconstraints = [InitAy @ y == init_colors,
               FinalAy @ y == final_colors,
               miniAy @ y == Aw @ w,
                y>=0, y<=1]

In [ ]:
cobj = np.ones((N))
ignorevalues = np.ravel([np.eye(n) for _ in range(p)])
cobj -= ignorevalues

objective = cvx.Minimize(cobj @ x)


constraints = [A @ x == b,
                 B @ x <= 1,
                 D @ x <= 1,
               x <= 1,
               x >= 0]

constraints.extend(wconstraints)
constraints.extend(yconstraints)

prob = cvx.Problem(objective, constraints)

result= prob.solve()

In [ ]:
print("Objective value :",prob.value)


xvalues = abs(
    np.reshape(
    np.round(x.value, 2),(p,n,n)))
print("\n X values : \n", xvalues)

yvalues = np.round(y.value, 2)


print("\n Y values : \n")

yvalues = abs(
    np.reshape(yvalues,(p,n,c)) )
print(yvalues)

Objective value : 10.006122449145742

 X values : 
 [[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]

 [[0.46 0.   0.   0.   0.   0.   0.   0.   0.   0

#### Backtracking

In [ ]:
def iter_res(objective, constraints): # THE ALGO

  # solve once
  prob = cvx.Problem(objective, constraints)
  result = prob.solve()
  print("Status at iteration 0", "is", prob.status, "\nwith optimal value", prob.value, "and", len(constraints), "constraints")

  # if value is >= 18.33333, then probably not good


  initCstrSize = len(constraints)
  currentCstrSize = len(constraints)
  # init branches
  branches = [(0, 0, initCstrSize)] # redundant branch
  iter = 0

  upperBound = float('inf')

  while True:
    iter += 1
    print("\n\niteration", iter)

    # get a kid from stack
    index, value, cstrSize = branches.pop()
    print("popped", index, "and", value, "and", cstrSize, "constraints")
    fixCstr = np.zeros(N)
    fixCstr[index] = 1
    constraints = constraints[:cstrSize] # keep appropriate part of cstr
    constraints.append(fixCstr @ x == value)
    currentCstrSize = cstrSize + 1

    # solve
    print("about to solve with", len(constraints), "cstrs")
    problem = cvx.Problem(objective, constraints)
    result = problem.solve()
    print("seems", problem.status, "with value", problem.value)

    # if infeasible, backtrack, not in final state and cannot branch
    if problem.status == "infeasible" or problem.value > upperBound:
      print("it's the infeasible loop or too high value...")

    else:
      # check entries
      values = abs(
          np.reshape(
          np.round(x.value, 2),(p,n,n)))
      # print(values)
      values = np.ravel(values)
      print("here it is feasible")
      if iter % 3 == 0: # printing out every third fo denbuggig
        xvalues = abs(
            np.reshape(
            np.round(x.value, 2),(p,n,n)))

        yvalues = abs(
            np.reshape(
            np.round(y.value, 2),(p,n,c)))

        print("X values :\n",xvalues)

        print("Y values :\n",yvalues)


      # index = check_entries(values)
      index = - 1
      for i in range(len(values)):
        if within_epsilon(values[i], 0) or within_epsilon(values[i], 1):
          continue
        else:
          index = i
          break

      if index == -1:
        # all good, goal state
        return x, y
        break
      else:
        # new node state
        # add a kids nodes to queue
        branches += [(i, 1, currentCstrSize), (i, 0, currentCstrSize)]
        print(branches)


# NOTES
# 208 -> 1 since 208->0 not feasible
# then
# 419 -> 1 otherwise bad


In [ ]:
np.set_printoptions(threshold=sys.maxsize)
cobj = np.ones((N))
ignorevalues = np.ravel([np.eye(n) for _ in range(p)])
cobj -= ignorevalues

objective = cvx.Minimize(cobj @ x)

constraints = [A @ x == b,
                B @ x <= 1,
                D @ x <= 1,
              x <= 1,
              x >= 0]

constraints2 = constraints.copy()
constraints2.extend(wconstraints)
constraints2.extend(yconstraints)

iter_res(objective, constraints2)

"""
for index in [208, 209, 221, 419]: # 420, 432
  fixCstr = np.zeros(N)
  fixCstr[index] = 1
  constraints2.append(fixCstr @ x == 0)

fixCstr = np.zeros(N)
fixCstr[420] = 1
constraints2.append(fixCstr @ x == 1)
fixCstr = np.zeros(N)
fixCstr[432] = 1
constraints2.append(fixCstr @ x == 1)
# mamu ti jebem
"""

xvalues = abs(
    np.reshape(
    np.round(x.value, 2),(p,n,n)))

yvalues = abs(
    np.reshape(
    np.round(y.value, 2),(p,n,c)))

print("X values :\n",xvalues)

print("Y values :\n",yvalues)

## Presidency constraint

In [ ]:
graph13 = [
    #Wing N edges
    [0,1],
    [1,2],
    [2,4],
    [0,7],

    #Wing B edges
    [3,4],
    [4,5],
    [3,10],
    [5,12],

    #Wing P edges
    [6,7],
    [7,8],
    [6,9],
    [8,11],

    #Wing C edges
    [9,10],

    #Wing A edges
    [11,12]
]

nb_edges = len(graph13)

PresidencyConstraints = np.empty((2*nb_edges*(p-1), N2))
for k in range(1,p-1):
  edgecount=0
  for edge in graph13:
    cst1 = np.zeros(N2).reshape((p,n,c))
    cst1[k,edge[0], 3], cst1[k,edge[1], 4] = 1,1
    PresidencyConstraints[(k-1)*2*nb_edges + 2*edgecount] = cst1.ravel()

    cst2 = np.zeros(N2).reshape((p,n,c))
    cst1[k,edge[1], 3], cst1[k,edge[0], 4] = 1,1
    PresidencyConstraints[(k-1)*2*nb_edges + 2*edgecount +1] = cst2.ravel()

    edgecount+=1

In [ ]:
constraints_president = constraints.copy()

constraints_president.append(PresidencyConstraints @ y <= 1)

prob = cvx.Problem(objective, constraints_president)

result= prob.solve()

In [ ]:
yvalues = np.round(y.value, 2)


print("\n Y values : \n")

yvalues = abs(
    np.reshape(yvalues,(p,n,c)) )
print(yvalues)

xvalues = abs(
    np.reshape(
    np.round(x.value, 2),(p,n,n)))
print("\n X values : \n", xvalues)



TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

# Quadratic programming

In [ ]:
zf = np.block([
    final_colors, final_colors, final_colors, final_colors
])

zi = np.block([init_colors for _ in range(4)])

In [ ]:
lambbda = 1
objective2 = cvx.Minimize(cobj @ x +
                         lambbda * cvx.sum_squares(zf - y[n*c : 5*n*c]))


prob = cvx.Problem(objective2, constraints)

result= prob.solve(solver = cvx.ECOS)

In [ ]:
lambbda = 100
objective2 = cvx.Minimize(cobj @ x +
                         lambbda * cvx.sum_squares(zf - y[n*c : 5*n*c]))


prob = cvx.Problem(objective2, constraints)

result= prob.solve(solver = cvx.ECOS)

In [ ]:
xvalues = abs(
    np.reshape(
    np.round(x.value, 2),(p,n,n)))
print("\n X values : \n", xvalues)

yvalues = np.round(y.value, 2)


print("\n Y values : \n")

yvalues = abs(
    np.reshape(yvalues,(p,n,c)) )
print(yvalues)


 X values : 
 [[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.  ]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]

 [[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.  

### Question 9

In [ ]:
zi = np.block([init_colors for _ in range(4)])

In [ ]:
lambbda = 100
objective2 = cvx.Minimize(cobj @ x +
                         lambbda * cvx.sum_squares(zi - y[n*c : 5*n*c]))


prob = cvx.Problem(objective2, constraints)

result= prob.solve(solver = cvx.ECOS)

In [ ]:
xvalues = abs(
    np.reshape(
    np.round(x.value, 2),(p,n,n)))
print("\n X values : \n", xvalues)

yvalues = np.round(y.value, 2)


print("\n Y values : \n")

yvalues = abs(
    np.reshape(yvalues,(p,n,c)) )
print(yvalues)